In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

CEM_TYPE = ["cem_type_CEM B", "cem_type_CEM C"]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [4]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical + One-Hot",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [5]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [6]:
df_copy = df.copy()
df_copy[CEM_TYPE] = df_copy[CEM_TYPE].astype(int)
df_copy = df.drop(
    [
        # Properties
        "Blaine",
        "ph2oimm", # Maybe pH of the immersion liquid | pH (acidity or alkalinity) 
        "Initial Setting Time",
        "flow", # Maybe flow table test
        "residuo 24 micron", # Maybe the residue left after passing the material through a sieve
        "R_wp", # Maybe water to powder ratio
        "Soundness",
        
        "CS2", # 2-day Compressive Strength
        
        # Chemical Composition
        # Reason: Loss on Ignition is the only feature
        # that belongs to chemical composition in which was 
        # measured by a different method, namely manual        
    ],
    axis=1,
).copy()


<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [7]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [8]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.413 (0.022)
MAE: -1.950 (0.022)
MAPE: -0.040 (0.000)
R2: 0.867 (0.003)


******
[TEST]
******
RMSE: -2.503 (0.088)
MAE: -2.026 (0.072)
MAPE: -0.042 (0.002)
R2: 0.856 (0.011)




<IPython.core.display.Javascript object>

In [9]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model Cross Validation  \
                                                                            
0  Chemical + Mineralogical + One-Hot  Linear Regression   Repeated KFold   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  2.502552  0.088184  2.026478  0.071808  0.041714  0.001673  0.855801   

             
        std  
0  0.011147

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [10]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.936 (0.262)
MAE: -1.544 (0.198)
MAPE: -0.032 (0.004)
R2: 0.910 (0.025)


******
[TEST]
******
RMSE: -3.409 (1.307)
MAE: -2.550 (0.956)
MAPE: -0.051 (0.017)
R2: 0.668 (0.220)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                           
0  Chemical + Mineralogical + One-Hot  Linear Regression   
1  Chemical + Mineralogical + One-Hot  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.408762  1.307381  2.550015  0.956150   
1              Repeated KFold  2.502552  0.088184  2.026478  0.071808   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.051231  0.016828  0.667876  0.219631  
1  0.041714  0.001673  0.855801  0.011147

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [12]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -2.375 (0.076)
MAE: -1.884 (0.080)
MAPE: -0.039 (0.001)
R2: 0.868 (0.007)


******
[TEST]
******
RMSE: -3.390 (0.902)
MAE: -2.370 (0.313)
MAPE: -0.049 (0.007)
R2: 0.714 (0.136)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                           
0  Chemical + Mineralogical + One-Hot  Linear Regression   
1  Chemical + Mineralogical + One-Hot  Linear Regression   
2  Chemical + Mineralogical + One-Hot  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.408762  1.307381  2.550015  0.956150   
1              Repeated KFold  2.502552  0.088184  2.026478  0.071808   
2           Time Series Split  3.390036  0.902384  2.369639  0.312721   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.051231  0.016828  0.667876  0.219631  
1  0.041714  0.001673  0.855801  0.011147  
2  0.048827  0.006726  0.714040  0.136428

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [14]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 2.384 (0.000)
MAE: 1.909 (0.000)
MAPE: 0.039 (0.000)
R2: 0.873 (0.000)


******
[TEST]
******
RMSE: 2.820 (0.000)
MAE: 2.271 (0.000)
MAPE: 0.047 (0.000)
R2: 0.797 (0.000)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                           
0  Chemical + Mineralogical + One-Hot  Linear Regression   
1  Chemical + Mineralogical + One-Hot  Linear Regression   
2  Chemical + Mineralogical + One-Hot  Linear Regression   
3  Chemical + Mineralogical + One-Hot  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.408762  1.307381  2.550015  0.956150   
1           Out of time Split  2.820190  0.000000  2.270981  0.000000   
2              Repeated KFold  2.502552  0.088184  2.026478  0.071808   
3           Time Series Split  3.390036  0.902384  2.369639  0.312721   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.051231  0.016828  0.667876  0.219631  
1  0.046923  0.000000  0.796692  0.000000  
2  0.041714  0.001673  0.855801  0.011147  
3  0.048827  0.006726  0.714040  0.136428

<IPython.core.display.Javascript object>

In [16]:
pipeline.named_steps["estimator"].coef_

array([ 3.81697957e-01, -4.30317739e-01, -1.59528441e+01, -9.67931033e+00,
        3.53873318e+01, -1.69147921e+00,  3.31361321e-02,  5.16199949e+00,
       -2.77499955e+00, -5.07807692e+00,  7.67990466e+00,  3.57576452e+00,
        1.27897528e+01,  1.04077226e+01, -1.29986306e+01,  4.52547247e-01,
        1.24702328e+00,  8.02646911e-01,  8.51700422e-01,  3.94397811e-01,
        2.95424722e+00,  2.07969866e+00,  3.14471759e+00,  1.90488952e+01,
       -9.38553142e-01,  5.39650931e-01, -9.26308892e-01, -5.08037751e+00])

<IPython.core.display.Javascript object>

In [17]:
pipeline.named_steps["estimator"].intercept_

49.22584856396868

<IPython.core.display.Javascript object>

In [18]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['SO3', 'Cl-', 'Alite_M3 C3S M3', 'Alite_M1 C3S M1', 'Alite_Sum C3S tot',
       'Ratio_M1 (rapporto M1/M3)', 'C3S_CS (taglia dei cristalliti C3S)',
       'Belite_beta', 'C3A cub', 'C3A_ortho', 'C3A tot', 'C4AF', 'CaO',
       'Ca(OH)2', 'Calce libera', 'Periclasio (MgO)', 'Quartz', 'K2SO4',
       'Langbeinite – MgK2(SO4)2', 'Aphthitalite – (K,Na)3(SO4)2', 'Gesso',
       'Emiidrato', 'Anidrite', 'Calcite – CaCO3', 'SO3_XRD', 'CO2_XRD',
       'cem_type_CEM B', 'cem_type_CEM C'],
      dtype='object')

<IPython.core.display.Javascript object>

In [20]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(28,)

<IPython.core.display.Javascript object>

In [21]:
x.columns.shape

(28,)

<IPython.core.display.Javascript object>

In [22]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [23]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
Alite_Sum C3S tot,35.387332
Calcite – CaCO3,19.048895
CaO,12.789753
Ca(OH)2,10.407723
C3A tot,7.679905
Belite_beta,5.161999
C4AF,3.575765
Anidrite,3.144718
Gesso,2.954247
Emiidrato,2.079699


<IPython.core.display.Javascript object>

In [24]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                           
0  Chemical + Mineralogical + One-Hot  Linear Regression   
1  Chemical + Mineralogical + One-Hot  Linear Regression   
2  Chemical + Mineralogical + One-Hot  Linear Regression   
3  Chemical + Mineralogical + One-Hot  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  3.408762  1.307381   
1                                {"Test Size": 0.2}  2.820190  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  2.502552  0.088184   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  3.390036  0.902384   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  2.550015  0.956150  0.051231  0.016828  0.667876  0.219631  
1  2.270981  0.000000  0.046923  0.000000  0.796692  0.000000  
2  2.026478  0.071808  0.041714  0.001673  0.855801  0.011147  
3  2.369639  0.312721  0.048827  0.006726  0.714040  0.136428

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [25]:
path = "../../../../../reports/results/local_models/partner_iv/all_cements/full/"
filename = "linear_regression_results_full_2.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [26]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/all_cements/grouped/"
filename = "linear_regression_results_grouped_2.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [27]:
df_results_to_save

Category Company Data Shape Timesteps  \
                                              
0  Local Model   partner_iv  (958, 28)       NaN   
1  Local Model   partner_iv  (958, 28)       NaN   
2  Local Model   partner_iv  (958, 28)       NaN   
3  Local Model   partner_iv  (958, 28)       NaN   

                             Features              Model  \
                                                           
0  Chemical + Mineralogical + One-Hot  Linear Regression   
1  Chemical + Mineralogical + One-Hot  Linear Regression   
2  Chemical + Mineralogical + One-Hot  Linear Regression   
3  Chemical + Mineralogical + One-Hot  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   1.936087  0.261538  ...   
1                                {"Test Size": 0.2}   2.383574  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   2.412731  0.022175  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   2.374816  0.075887  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.909941  0.025086  3.408762  1.307381  2.550015  0.956150  0.051231   
1  0.873020  0.000000  2.820190  0.000000  2.270981  0.000000  0.046923   
2  0.866880  0.002697  2.502552  0.088184  2.026478  0.071808  0.041714   
3  0.868435  0.007475  3.390036  0.902384  2.369639  0.312721  0.048827   

              R2 Test            
        std      mean       std  
0  0.016828  0.667876  0.219631  
1  0.000000  0.796692  0.000000  
2  0.001673  0.855801  0.011147  
3  0.006726  0.714040  0.136428  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>

In [28]:
c = pd.read_csv(
    "../../../../../reports/results/local_models/partner_i-oficial/all_cements/grouped/linear_regression_results_grouped_2.csv",
    header=[0, 1],
).rename(columns=lambda x: "" if "Unnamed" in x else x, level=1)

<IPython.core.display.Javascript object>

In [29]:
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (Category, )                 4 non-null      object 
 1   (Company, )                  4 non-null      object 
 2   (Data Shape, )               4 non-null      object 
 3   (Timesteps, )                0 non-null      float64
 4   (Features, )                 4 non-null      object 
 5   (Model, )                    4 non-null      object 
 6   (Cross Validation, )         4 non-null      object 
 7   (Cross Validation Params, )  4 non-null      object 
 8   (RMSE Train, mean)           4 non-null      float64
 9   (RMSE Train, std)            4 non-null      float64
 10  (MAE Train, mean)            4 non-null      float64
 11  (MAE Train, std)             4 non-null      float64
 12  (MAPE Train, mean)           4 non-null      float64
 13  (MAPE Train, std)       

<IPython.core.display.Javascript object>

In [30]:
df_results_to_save.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (Category, )                 4 non-null      object 
 1   (Company, )                  4 non-null      object 
 2   (Data Shape, )               4 non-null      object 
 3   (Timesteps, )                0 non-null      float64
 4   (Features, )                 4 non-null      object 
 5   (Model, )                    4 non-null      object 
 6   (Cross Validation, )         4 non-null      object 
 7   (Cross Validation Params, )  4 non-null      object 
 8   (RMSE Train, mean)           4 non-null      float64
 9   (RMSE Train, std)            4 non-null      float64
 10  (MAE Train, mean)            4 non-null      float64
 11  (MAE Train, std)             4 non-null      float64
 12  (MAPE Train, mean)           4 non-null      float64
 13  (MAPE Train, std)       

<IPython.core.display.Javascript object>

In [31]:
df_results_to_save[df_results_to_save["Cross Validation Params"].str.contains("Date")]

Empty DataFrame
Columns: [(Category, ), (Company, ), (Data Shape, ), (Timesteps, ), (Features, ), (Model, ), (Cross Validation, ), (Cross Validation Params, ), (RMSE Train, mean), (RMSE Train, std), (MAE Train, mean), (MAE Train, std), (MAPE Train, mean), (MAPE Train, std), (R2 Train, mean), (R2 Train, std), (RMSE Test, mean), (RMSE Test, std), (MAE Test, mean), (MAE Test, std), (MAPE Test, mean), (MAPE Test, std), (R2 Test, mean), (R2 Test, std)]
Index: []

[0 rows x 24 columns]

<IPython.core.display.Javascript object>

In [32]:
df.shape

(958, 38)

<IPython.core.display.Javascript object>